In [1]:
from torchvision import models
import os
from torch.autograd import Variable
import torch
import cv2
import numpy as np

# Load the pretrained DNN

In [2]:
ResNet50 = models.resnet50(pretrained=True)

# Convert the DNN to ONNX mat

In [3]:
# define the directory for further converted model save
onnx_model_path = "models"
# define the name of further converted model
onnx_model_name = "resnet50.onnx"

# create directory for further converted model
os.makedirs(onnx_model_path, exist_ok=True)

# get full path to the converted model
full_model_path = os.path.join(onnx_model_path, onnx_model_name)

# generate model input
generated_input = Variable(
    torch.randn(1, 3, 224, 224)
)

# model export into ONNX format
torch.onnx.export(
    ResNet50,
    generated_input,
    full_model_path,
    verbose=True,
    input_names=["star"],
    output_names=["output"],
    opset_version=11
)

graph(%star : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %fc.weight : Float(1000, 2048, strides=[2048, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %497 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %498 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %500 : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %501 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %503 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %504 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %506 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %507 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %509 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %510 : Float(256, strides=[1], require

# Using opencv load ONNX mat

In [4]:
opencv_net = cv2.dnn.readNetFromONNX('./models/resnet50.onnx')
print("OpenCV model was successfully read. Layer IDs: \n", opencv_net.getLayerNames())

OpenCV model was successfully read. Layer IDs: 
 ('onnx_node!Conv_0', 'onnx_node!Relu_1', 'onnx_node!MaxPool_2', 'onnx_node!Conv_3', 'onnx_node!Relu_4', 'onnx_node!Conv_5', 'onnx_node!Relu_6', 'onnx_node!Conv_7', 'onnx_node!Conv_8', 'onnx_node!Add_9', 'onnx_node!Relu_10', 'onnx_node!Conv_11', 'onnx_node!Relu_12', 'onnx_node!Conv_13', 'onnx_node!Relu_14', 'onnx_node!Conv_15', 'onnx_node!Add_16', 'onnx_node!Relu_17', 'onnx_node!Conv_18', 'onnx_node!Relu_19', 'onnx_node!Conv_20', 'onnx_node!Relu_21', 'onnx_node!Conv_22', 'onnx_node!Add_23', 'onnx_node!Relu_24', 'onnx_node!Conv_25', 'onnx_node!Relu_26', 'onnx_node!Conv_27', 'onnx_node!Relu_28', 'onnx_node!Conv_29', 'onnx_node!Conv_30', 'onnx_node!Add_31', 'onnx_node!Relu_32', 'onnx_node!Conv_33', 'onnx_node!Relu_34', 'onnx_node!Conv_35', 'onnx_node!Relu_36', 'onnx_node!Conv_37', 'onnx_node!Add_38', 'onnx_node!Relu_39', 'onnx_node!Conv_40', 'onnx_node!Relu_41', 'onnx_node!Conv_42', 'onnx_node!Relu_43', 'onnx_node!Conv_44', 'onnx_node!Add_45

# preprocess image
加载标签
处理图像

## 加载标签

In [5]:
with open('./data/dnn/classification_classes_ILSVRC2012.txt') as f:
    imagenet_labels = [line.strip() for line in f.readlines()]
print(imagenet_labels)

['tench, Tinca tinca', 'goldfish, Carassius auratus', 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias', 'tiger shark, Galeocerdo cuvieri', 'hammerhead, hammerhead shark', 'electric ray, crampfish, numbfish, torpedo', 'stingray', 'cock', 'hen', 'ostrich, Struthio camelus', 'brambling, Fringilla montifringilla', 'goldfinch, Carduelis carduelis', 'house finch, linnet, Carpodacus mexicanus', 'junco, snowbird', 'indigo bunting, indigo finch, indigo bird, Passerina cyanea', 'robin, American robin, Turdus migratorius', 'bulbul', 'jay', 'magpie', 'chickadee', 'water ouzel, dipper', 'kite', 'bald eagle, American eagle, Haliaeetus leucocephalus', 'vulture', 'great grey owl, great gray owl, Strix nebulosa', 'European fire salamander, Salamandra salamandra', 'common newt, Triturus vulgaris', 'eft', 'spotted salamander, Ambystoma maculatum', 'axolotl, mud puppy, Ambystoma mexicanum', 'bullfrog, Rana catesbeiana', 'tree frog, tree-frog', 'tailed frog, bell toad, 

# 处理图像

In [6]:
# 加载并处理图像
input_img = cv2.imread('./wang.jpeg', cv2.IMREAD_COLOR)
input_img = input_img.astype(np.float32)

input_img = cv2.resize(input_img, (256, 256))

# define preprocess parameters
mean = np.array([0.485, 0.456, 0.406]) * 255.0
scale = 1 / 255.0
std = [0.229, 0.224, 0.225]

# prepare input blob to fit the model input:
# 1. subtract mean
# 2. scale to set pixel values from 0 to 1
input_blob = cv2.dnn.blobFromImage(
    image=input_img,
    scalefactor=scale,
    size=(224, 224),  # img target size
    mean=mean,
    swapRB=True,  # BGR -> RGB
    crop=True  # center crop
)
# 3. divide by std
input_blob[0] /= np.asarray(std, dtype=np.float32).reshape(3, 1, 1) # 只让通道标准化

preproc_img = input_blob

# predict using DNN

In [7]:
# 进行预测
# set OpenCV DNN input
opencv_net.setInput(preproc_img)

# OpenCV DNN inference
out = opencv_net.forward()
print("OpenCV DNN prediction: \n")
print("* shape: ", out.shape)   # (1,1000)

# get the predicted class ID  获取最大的概率的index
imagenet_class_id = np.argmax(out)

# get confidence 预测为该分类的概率
confidence = out[0][imagenet_class_id]

print("* class ID: {}, label: {}".format(imagenet_class_id, imagenet_labels[imagenet_class_id]))
print("* confidence: {:.4f}".format(confidence))

OpenCV DNN prediction: 

* shape:  (1, 1000)
* class ID: 655, label: miniskirt, mini
* confidence: 10.4047
